In [3]:
import pandas as pd
df = pd.read_json("../task_1/data/fragments_classification.jsonl", lines=True)
df

,text,label
0,Nie uzna gola. Robben był kilka metrów w polu ...,"[[0, 8, odwrócenie]]"
1,@USER No właśnie o tym jest ten tweet 😄,[]
2,@USER @USER Widać chcą wiecej polskich mord go...,"[[23, 38, wzmocnienie]]"
3,"Idę spać bo padam na twarz, w końcu w domuuuu",[]
4,@USER Tak się poznałam z moim chłopakiem 😂 cza...,[]
...,...,...
795,@USER Wszystkiego najlepszego z okazji urodzin...,"[[5, 29, wzmocnienie]]"
796,"@USER widzę, że pewne tweety działają jak magn...",[]
797,"@USER @USER Chociaż futro ma z jenota,\nTo nie...","[[43, 52, odwrócenie], [55, 67, wzmocnienie], ..."
798,@USER Ty aby nie zacząleś ćpać przez wydumane ...,"[[13, 25, odwrócenie], [37, 54, wzmocnienie]]"


In [14]:
from transformers import AutoTokenizer

In [15]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [20]:
def get_annotated_fragments(labels: list):
    frag_dict = {}
    for label in labels:
        frag_dict[(label[0], label[1])] = label[2]
    return frag_dict

In [25]:
a = get_annotated_fragments(df['label'][798])
a

{(13, 25): 'odwrócenie', (37, 54): 'wzmocnienie'}

In [34]:
encoding = tokenizer(df['text'][0], return_offsets_mapping=True)
annotated_fragments = get_annotated_fragments(df['label'][0])
# Check tokens against annotated fragments
tokens_in_annotations = []
for idx, (start, end) in enumerate(encoding['offset_mapping']):
    token_text = encoding['input_ids'][idx]
    for fragment_start, fragment_end in annotated_fragments:
        # Check if token overlaps with the fragment
        if start < fragment_end and end > fragment_start:
            tokens_in_annotations.append(token_text)  # decode token id to string
            print(annotated_fragments[(fragment_start, fragment_end)])

print("Tokens in annotated fragments:", tokens_in_annotations)

odwrócenie
odwrócenie
odwrócenie
odwrócenie
odwrócenie
Tokens in annotated fragments: [9152, 2063, 1057, 2480, 2532]


In [35]:
encoding

{'input_ids': [101, 9152, 2063, 1057, 2480, 2532, 2175, 2721, 1012, 26211, 2368, 2011, 18818, 11382, 26518, 6005, 2860, 1059, 14955, 2226, 10556, 6826, 24335, 1010, 1059, 22571, 22648, 21293, 18818, 17540, 2666, 4241, 4143, 10975, 4371, 4213, 3351, 2079, 2079, 16313, 3211, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 2), (2, 3), (4, 5), (5, 6), (6, 8), (9, 11), (11, 13), (13, 14), (15, 19), (19, 21), (22, 24), (24, 25), (26, 28), (28, 31), (32, 37), (37, 38), (39, 40), (41, 44), (44, 45), (46, 48), (48, 50), (50, 52), (52, 53), (54, 55), (55, 57), (57, 60), (60, 63), (63, 64), (65, 68), (68, 70), (71, 73), (73, 75), (76, 78), (78, 80), (80, 82), (82, 84), (85, 87), (88, 90), (90, 93), (93, 95), 

In [ ]:
encoding = tokenizer(df['text'][798], return_offsets_mapping=True)
annotated_fragments = get_annotated_fragments(df['label'][798])
# Check tokens against annotated fragments
tokens_in_annotations = []
for idx, (start, end) in enumerate(encoding['offset_mapping']):
    token_text = encoding['input_ids'][idx]
    for fragment_start, fragment_end in annotated_fragments:
        # Check if token overlaps with the fragment
        if start < fragment_end and end > fragment_start:
            tokens_in_annotations.append((token_text, annotated_fragments[(fragment_start, fragment_end)]))

print("Tokens in annotated fragments:", tokens_in_annotations)

Tokens in annotated fragments: [(9152, 'odwrócenie'), (2063, 'odwrócenie'), (29250, 'odwrócenie'), (16739, 'odwrócenie'), (2229, 'odwrócenie'), (1059, 'wzmocnienie'), (25688, 'wzmocnienie'), (19042, 'wzmocnienie'), (2063, 'wzmocnienie'), (3291, 'wzmocnienie'), (2100, 'wzmocnienie')]


In [39]:
def process_row(row):
    encoding = tokenizer(row['text'], return_offsets_mapping=True)
    annotated_fragments = get_annotated_fragments(row['label'])
    # Check tokens against annotated fragments
    tokens_in_annotations = []
    for idx, (start, end) in enumerate(encoding['offset_mapping']):
        token_text = encoding['input_ids'][idx]
        for fragment_start, fragment_end in annotated_fragments:
            # Check if token overlaps with the fragment
            if start < fragment_end and end > fragment_start:
                tokens_in_annotations.append((token_text, annotated_fragments[(fragment_start, fragment_end)]))
    return tokens_in_annotations

In [40]:
df['tokens_in_annotations_with_labels'] = df.apply(process_row, axis=1)

In [45]:
df['tokens_in_annotations_with_labels'][798]

[(9152, 'odwrócenie'),
 (2063, 'odwrócenie'),
 (29250, 'odwrócenie'),
 (16739, 'odwrócenie'),
 (2229, 'odwrócenie'),
 (1059, 'wzmocnienie'),
 (25688, 'wzmocnienie'),
 (19042, 'wzmocnienie'),
 (2063, 'wzmocnienie'),
 (3291, 'wzmocnienie'),
 (2100, 'wzmocnienie')]